In [1]:
# Import necessary libraries
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset

In [2]:
# Ensure CUDA is available for training
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
import torch, gc

# 1. Delete unneeded variables (e.g., models, tensors)
try:
    del model
except NameError:
    pass

# 2. Run Python's garbage collector
gc.collect()

# 3. Clear the PyTorch cache
torch.cuda.empty_cache()


In [4]:
# Login to Hugging Face to access the model (if necessary)
from huggingface_hub import notebook_login
notebook_login()


In [5]:
# Load the tokenizer and model from Hugging Face
model_checkpoint = "microsoft/Phi-4-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, use_auth_token=True).to(device)

/home/cse/anaconda3/envs/voipvishing/lib/python3.13/site-packages/transformers/models/auto/tokenization_auto.py:862: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/cse/anaconda3/envs/voipvishing/lib/python3.13/site-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# Define the path to the CSV dataset
csv_path = "/home/cse/Desktop/VOIP_VISHING/REPOS/conversation-prediction/FINAL_DATASET2.csv"

In [7]:
# Function to load and preprocess data
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath)
    grouped = df.groupby('CONVERSATION_ID').agg({'TEXT': ' '.join}).reset_index()
    grouped['partial_text'] = grouped['TEXT'].apply(lambda x: " ".join(x.split()[:int(len(x.split()) * 0.5)]))
    grouped['full_text'] = grouped['TEXT']
    return grouped[['partial_text', 'full_text']]

In [8]:

data = load_and_preprocess_data(csv_path)


In [9]:
(data.head())


,partial_text,full_text
0,"Hello, this is [Your Name]'s personal assistan...","Hello, this is [Your Name]'s personal assistan..."
1,"Hello, this is the personal assistant of [Your...","Hello, this is the personal assistant of [Your..."
2,"Hello, this is the assistant to [Your Name]. H...","Hello, this is the assistant to [Your Name]. H..."
3,"Hello, this is the office of [Your Name]. How ...","Hello, this is the office of [Your Name]. How ..."
4,"Hello, you've reached the assistant for [Your ...","Hello, you've reached the assistant for [Your ..."


In [10]:
# Prepare dataset for training
def tokenize_function(examples):
    model_inputs = tokenizer(examples['partial_text'], max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['full_text'], max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    model_inputs['labels'] = labels['input_ids'].squeeze()
    return model_inputs

In [11]:
dataset = Dataset.from_pandas(data)
hf_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/76 [00:00<?, ? examples/s]

/home/cse/anaconda3/envs/voipvishing/lib/python3.13/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [12]:
train_dataset, eval_dataset = hf_dataset.train_test_split(test_size=0.2).values()


In [13]:

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./model_finetuned',
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

/home/cse/anaconda3/envs/voipvishing/lib/python3.13/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

/tmp/ipykernel_91430/311847730.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
# Train the model
trainer.train()


OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB. GPU 0 has a total capacity of 15.69 GiB of which 10.62 MiB is free. Including non-PyTorch memory, this process has 15.62 GiB memory in use. Of the allocated memory 15.37 GiB is allocated by PyTorch, and 11.78 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)